<a href="https://colab.research.google.com/github/Nerikoutchala/Nerikoutchala/blob/main/PESO_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Importing libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

### Step 2: Defining the U-Net model

In [26]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.down_conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.down_conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.down_conv3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.down_conv4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up_conv1 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up_conv2 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.up_conv3 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        x1 = self.down_conv1(x)
        x2 = self.down_conv2(x1)
        x3 = self.down_conv3(x2)
        x4 = self.down_conv4(x3)

        y = self.up_conv1(x4)
        y = torch.cat([x3, y], dim=1)
        y = self.up_conv2(y)
        y = torch.cat([x2, y], dim=1)
        y = self.up_conv3(y)
        y = torch.cat([x1, y], dim=1)
        y = self.final_conv(y)

        return y

### Step 3: Defining the loss function and optimizer

In [27]:
model = UNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Step 4: Defining the data loader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataset, targets, transform=None):
        self.dataset = dataset
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, _ = self.dataset[idx]
        target = self.targets[idx]
        
        if self.transform:
            img = self.transform(img)

        return img, target

    
    
data_dir = "/content/drive/MyDrive/peso/"


# Define the transforms to be applied on the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])




# Load the dataset using ImageFolder
dataset = ImageFolder(data_dir, transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])


#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
#test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the data loaders
train_dataset = ImageFolder(data_dir, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create the MyDataset objects
train_dataset = MyDataset(train_dataset, train_dataset.targets, transform=transforms.ToTensor())
val_dataset = MyDataset(val_dataset.dataset, val_dataset.dataset.targets, transform=transforms.ToTensor())
test_dataset = MyDataset(test_dataset.dataset, test_dataset.dataset.targets, transform=transforms.ToTensor())

# Define the data loaders for the MyDataset objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




# Get an example batch of data
x, y = next(iter(train_loader))

# Print the shape of the input tensor
print(x.shape)

# Print the shape of the label tensor
print(y.shape)

### Step 5: Training the model

In [ ]:
epochs = 10

for epoch in range(epochs):
    train_loss = 0.0
    val_loss = 0.0

    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()

        y_pred = model(x)
        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)

    model.eval()
    with torch.no_grad():
        for x, y in val_loader:
            y_pred = model(x)
            loss = criterion(y_pred, y)

            val_loss += loss.item() * x.size(0)

    train_loss /= len(train_loader.dataset)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

### Step 6: Testing the model

In [ ]:
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        y_pred = model(x)
        loss = criterion(y_pred, y)

        print(f"Test Loss: {loss.item():.4f}")